In [ ]:
import pandas as pd
from sqlalchemy import create_engine, Table, MetaData

In [ ]:
pd.set_option('max_columns', 200)
pd.set_option('max_rows', 300)
pd.set_option('display.expand_frame_repr', True)

### Data Files Location

* Most data files for the exercises can be found on the [course site](#https://www.datacamp.com/courses/introduction-to-relational-databases-in-python)
    * [Census (CSV)](#https://assets.datacamp.com/production/repositories/274/datasets/7a5a4567430ee737c70994d1c4747f252e0fd527/census.csv)
    * [Census (SQLite)](#https://assets.datacamp.com/production/repositories/274/datasets/f6eda83e7fb90ac06a22af4132a355933763785c/census.sqlite)
    * [Employees (SQLite)](#https://assets.datacamp.com/production/repositories/274/datasets/af705f788c225cad7e6ef405ed5490db36ed03bf/employees.sqlite)    
* Other data files may be found in my [DataCamp repository](#https://github.com/trenton3983/DataCamp/tree/master/data)

### Data File Objects

In [ ]:
census_csv_data = 'data/intro_to_databases_in_python/census.csv'
census_sql_data = 'sqlite:///data/intro_to_databases_in_python/census.sqlite'
employees_sql_data = 'sqlite:///data/intro_to_databases_in_python/employees.sqlite'

# Introduction to Databases in Python

***Course Description***

In this Python SQL course, you'll learn the basics of using Structured Query Language (SQL) with Python. This will be useful since whether you like it or not, databases are ubiquitous and, as a data scientist, you'll need to interact with them constantly. The Python SQL toolkit SQLAlchemy provides an accessible and intuitive way to query, build & write to SQLite, MySQL and Postgresql databases (among many others), all of which you will encounter in the daily life of a data scientist.

## 1: Basics of Relational Databases

In this chapter, you will become acquainted with the fundamentals of Relational Databases and the Relational Model. You will learn how to connect to a database and then interact with it by writing basic SQL queries, both in raw SQL as well as with SQLAlchemy, which provides a Pythonic way of interacting with databases.

### 1.a: Introduction to Databases

#### A database consists of tables

![alt text](https://github.com/trenton3983/DataCamp/blob/master/Images/intro_to_databases_in_python/tables.JPG?raw=true "Tables")

#### Table consists of columns and rows

![alt text](https://github.com/trenton3983/DataCamp/blob/master/Images/intro_to_databases_in_python/columns_rows.JPG?raw=true "Columns and Rows")

#### Tables can be related

![alt text](https://github.com/trenton3983/DataCamp/blob/master/Images/intro_to_databases_in_python/related.JPG?raw=true "Related")

### Exercises

#### Relational Model

Which of the following is not part of the relational model?

Answer the question

1. Tables
2. Columns
3. Rows
4. __Dimensions__
5. Relationships


### 1.b: Connecting to a Database

#### Meet SQLAlchemy

* Two Main Pieces
    * Core (Relational Model focused)
    * ORM (User Data Model focused)
        * Object Relational Model

#### There are many types of databases

* SQLite
* PostgreSQL
* MySQL
* MS SQL
* Oracle
* Many more

#### Connecting to a database

```python
In [1]: from sqlalchemy import create_engine
In [2]: engine = create_engine('sqlite:///census_nyc.sqlite')
In [3]: connection = engine.connect()
```

* Engine: common interface to the database from SQLAlchemy
* Connection string: All the details required to find the database (and login, if necessary)

#### A word on connection strings
* 'sqlite:///census_nyc.sqlite'
* Driver+Dialect Filename

#### What’s in your database?

* Before querying your database, you’ll want to know what is in it: what the tables are, for example:

```python
In [1]: from sqlalchemy import create_engine
In [2]: engine = create_engine('sqlite:///census_nyc.sqlite')
In [3]: print(engine.table_names())
Out[3]: ['census', 'state_fact']
```

#### Reflection

* Reflection reads database and builds SQLAlchemy Table objects

```python
In [1]: from sqlalchemy import MetaData, Table
In [2]: metadata = MetaData()
In [3]: census = Table('census', metadata, autoload=True, autoload_with=engine)
In [4]: print(repr(census))
Out[4]:
Table('census', MetaData(bind=None), Column('state',
VARCHAR(length=30), table=<census>), Column('sex',
VARCHAR(length=1), table=<census>), Column('age', INTEGER(),
table=<census>), Column('pop2000', INTEGER(), table=<census>),
Column('pop2008', INTEGER(), table=<census>), schema=None)
```

### Exercises

#### Engines and Connection Strings

Alright, it's time to create your first engine! An engine is just a common interface to a database, and the information it requires to connect to one is contained in a connection string, such as **sqlite:///census_nyc.sqlite**. Here, **sqlite** is the database driver, while **census_nyc.sqlite** is a SQLite file contained in the local directory.

You can learn a lot more about connection strings in the [SQLAlchemy documentation](#http://docs.sqlalchemy.org/en/latest/core/engines.html#database-urls).

Your job in this exercise is to create an engine that connects to a local SQLite file named **census.sqlite**. Then, print the names of the tables it contains using the **.table_names()** method. Note that when you just want to print the table names, you do not need to use **engine.connect()** after creating the engine.

**Instructions**

* Import **create_engine** from the **sqlalchemy** module.
* Using the **create_engine()** function, create an engine for a local file named **census.sqlite** with **sqlite** as the driver. Be sure to enclose the connection string within quotation marks.
* Print the output from the **.table_names()** method on the **engine**.

In [ ]:
# Import create_engine - at top of notebook

# Create an engine that connects to the census.sqlite file: engine
engine = create_engine(census_sql_data)

# Print table names
engine.table_names()

#### Autoloading Tables from a Database

SQLAlchemy can be used to automatically load tables from a database using something called reflection. Reflection is the process of reading the database and building the metadata based on that information. It's the opposite of creating a Table by hand and is very useful for working with existing databases. To perform reflection, you need to import the **Table** object from the SQLAlchemy package. Then, you use this **Table** object to read your table from the engine and autoload the columns. Using the **Table** object in this manner is a lot like passing arguments to a function. For example, to autoload the columns with the engine, you have to specify the keyword arguments **autoload=True** and **autoload_with=engine** to **Table()**.

In this exercise, your job is to reflect the **census** table available on your **engine** into a variable called **census**. The metadata has already been loaded for you using **MetaData()** and is available in the variable **metadata**.

**Instructions**

* Import the **Table** object from **sqlalchemy**.
* Reflect the **census** table by using the Table object with the arguments:
    * The name of the table as a string (**'census'**).
    * The metadata, contained in the variable **metadata**.
    * **autoload=True**
    * The engine to autoload with - in this case, **engine**.
* Print the details of **census** using the **repr()** function.

In [ ]:
# Import Table - at top of Notebook

# Reflect census table from the engine: census
metadata = MetaData()
census = Table('census', metadata, autoload=True, autoload_with=engine)

# Print census table metadata
repr(census)

#### Viewing Table Details

Great job reflecting the **census** table! Now you can begin to learn more about the columns and structure of your table. It is important to get an understanding of your database by examining the column names. This can be done by using the **.columns** attribute and accessing the **.keys()** method. For example, **census.columns.keys()** would return a list of column names of the **census** table.

Following this, we can use the metadata container to find out more details about the reflected table such as the columns and their types. For example, table objects are stored in the **metadata.tables** dictionary, so you can get the metadata of your **census** table with **metadata.tables['census']**. This is similar to your use of the **repr()** function on the census table from the previous exercise.

**Instructions**

* Reflect the **census** table as you did in the previous exercise using the **Table()** function.
* Print a list of column names of the **census** table by applying the **.keys()** method to **census.columns**.
* Print the details of the **census** table using the **metadata.tables** dictionary along with the **repr()** function. To do this, first access the **'census'** key of the **metadata.tables** dictionary, and place this inside the provided **repr()** function.

In [ ]:
# Reflect the census table from the engine: census
census = Table('census', metadata, autoload=True, autoload_with=engine)

# Print the column names
census.columns.keys()

In [ ]:
# Print full table metadata
repr(metadata.tables['census'])

### 1.c: Introduction to SQL

### Exercises

#### Selecting data from a Table: raw SQL

#### Selecting data from a Table with SQLAlchemy

#### Handling a ResultSet

## 2: Applying Filtering, Ordering and Grouping to Queries

In this chapter, you will build on the database knowledge you began acquiring in the previous chapter by writing more nuanced queries that allow you to filter, order, and count your data, all within the Pythonic framework provided by SQLAlchemy!

## 3: Advanced SQLAlchemy Queries

Herein, you will learn to perform advanced - and incredibly useful - queries that will enable you to interact with your data in powerful ways.

## 4: Creating and Manipulating your own Databases

In the previous chapters, you interacted with existing databases and queried them in various different ways. Now, you will learn how to build your own databases and keep them updated!

## 5: Putting it all together

Here, you will bring together all of the skills you acquired in the previous chapters to work on a real life project! From connecting to a database, to populating it, to reading and querying it, you will have a chance to apply all the key concepts you learned in this course.